In [11]:
import pandas as pd  
import numpy as np   
from bs4 import BeautifulSoup    
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초.

In [12]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 melon(음원사이트)
driver.get('http://www.melon.com')
time.sleep(0.5)

In [13]:
driver.find_element_by_css_selector(".menu_bg.menu04").click() #멜론DJ 클릭
time.sleep(0.5)
driver.find_element_by_css_selector(".menu_dj.m3").click() #테마장르 클릭
time.sleep(0.5)

tag_input = driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[1]/div[2]/div[2]/input")
tag_input.send_keys("행복")
driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[1]/div[2]/div[2]/button").click() # 검색 클릭

dict = {}    # 전체 크롤링 데이터 dictionary로 저장

for i in range(1,21):
    playlist_kind = "/html/body/div/div[3]/div/div/div[4]/div/ul/li["+str(i)+"]/div[2]/div[1]/a[2]"
    song_count = driver.find_element_by_css_selector("span.tot_song").text
    song_count = song_count.replace("곡","")
    driver.find_element_by_xpath(playlist_kind).click()
    time.sleep(0.5)
    next_page = False
    
    for j in range(int(song_count)):
        if(j > 50) :
            if(next_page == False) :
                driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a").click()
                next_page = True
            more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
            more_info_list[j-51].click()
        else :
            # 곡 한개씩 선택
            more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
            more_info_list[j].click()
        
        try : 
            music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
            time.sleep(0.5)
        
            # 제목 크롤링
            overlays = ".song_name"                                 
            tit = driver.find_element_by_css_selector(overlays)          # title
            title = tit.text

            # 가수 크롤링
            overlays = ".artist"                              
            nick = driver.find_element_by_css_selector(overlays)         # nickname
            artist_name = nick.text

            # 발매날짜 크롤링
            overlays = ".list"                                
            meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')  # datetime
            Album_date = meta_info[3]
        
            # 좋아요 수
            overlays = "d_like_count"
            like_count = driver.find_element_by_id(overlays)
            LIKE = like_count.text
        
            # 앨범이름 크롤링
            Album_name = meta_info[1]
        
            # 현재 곡 장르 크롤링
            Album_Genre = meta_info[5]
        
            # 더보기 버튼 클릭
            driver.find_element_by_css_selector(".button_more.arrow_d").click()
            time.sleep(0.5)
        
            # 가사 크롤링
            overlays = ".lyric.on"                                 
            contents = driver.find_elements_by_css_selector(overlays)    # contents
        
            content_list = []
            for content in contents:
                content_list.append(content.text)
 
            content_str = ' '.join(content_list)                         # content_str

            # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
            music_info['title'] = title
            music_info['artist'] = artist_name
            music_info['Album_date'] = Album_date
            music_info['LIKE'] = LIKE
            music_info['Album_name'] = Album_name
            music_info['Album_Genre'] = Album_Genre
            music_info['Lyric'] = content_str
            time.sleep(1)
            # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
            dict[i] = music_info
        
            # 글 하나 크롤링 후 뒤로가기
            driver.back()  
    
     # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
        except:
            driver.back()
            time.sleep(0.5)
            continue
        
        # 중간,중간에 파일로 저장하기  - 에러날 경우 데이터 손실 위험 감소
        if i%40 == 0:
        # 판다스로 만들기
            import pandas as pd
            result_df = pd.DataFrame.from_dict(dict, 'index')

            # 저장하기
            result_df.to_excel("Music_melon_happy_tag.xlsx", encoding='utf-8-sig')
            time.sleep(0.5)

# 저장하기
result_df.to_excel("Music_melon_happy_tag.xlsx", encoding='utf-8-sig')
time.sleep(0.5)
driver.quit()

AttributeError: 'NoneType' object has no attribute 'click'